<a href="https://colab.research.google.com/github/JasonGalante/nhanes_inferential_2023/blob/main/Jason_Galante_NHANES__pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
## Class Code
# Load Files Demographics
import pandas as pd
from scipy.stats import chi2_contingency

In [97]:
demo = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/DEMO_L.XPT', format='xport')
bp = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/BPXO_L.XPT', format='xport')
body = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/BMX_L.XPT', format='xport')
chol_total = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/TCHOL_L.XPT', format='xport')
glycohemo = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/GHB_L.XPT', format='xport')
crp = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/HSCRP_L.XPT', format='xport')
dm = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/DIQ_L.XPT', format='xport')

In [9]:
demographics = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/DEMO_L.XPT', format='xport')
demographics.DMDMARTZ

,DMDMARTZ
0,1.0
1,1.0
2,1.0
3,NaN
4,NaN
...,...
11928,NaN
11929,3.0
11930,1.0
11931,2.0


In [98]:
ms_status = demo[['DMDMARTZ']]
ms_status['DMDMARTZ'] = ms_status['DMDMARTZ'].astype(str)
ms_status.dtypes

ms_status['DMDMARTZ']



<ipython-input-98-55607a5755d6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ms_status['DMDMARTZ'] = ms_status['DMDMARTZ'].astype(str)


,DMDMARTZ
0,1.0
1,1.0
2,1.0
3,nan
4,nan
...,...
11928,nan
11929,3.0
11930,1.0
11931,2.0


In [99]:
ed_status = demo[['DMDEDUC2']]
ed_status['DMDEDUC2'] = ed_status['DMDEDUC2'].astype(str)
ed_status.dtypes

ed_status['DMDEDUC2']


<ipython-input-99-f390c418a69d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ed_status['DMDEDUC2'] = ed_status['DMDEDUC2'].astype(str)


,DMDEDUC2
0,5.0
1,5.0
2,3.0
3,nan
4,nan
...,...
11928,nan
11929,5.0
11930,4.0
11931,4.0


In [100]:
## check for null values DMDMARTZ
counts = pd.isna(demo['DMDMARTZ'])
counts.value_counts()

,count
DMDMARTZ,
False,7792
True,4141


In [101]:
## check for null values  DMDEDUC2
counts = pd.isna(demo['DMDEDUC2'])
counts.value_counts()

,count
DMDEDUC2,
False,7794
True,4139


In [102]:
# drop null values from DMDMARTZ and DMDEDUC2
demo_nonull= demo.dropna(subset=['DMDMARTZ', 'DMDEDUC2'])


demo_nonull[['DMDMARTZ' , 'DMDEDUC2']]






,DMDMARTZ,DMDEDUC2
0,1.0,5.0
1,1.0,5.0
2,1.0,3.0
6,3.0,2.0
7,1.0,3.0
...,...,...
11927,2.0,1.0
11929,3.0,5.0
11930,1.0,4.0
11931,2.0,4.0


In [103]:
# remove unwanted values
demo_nonull = demo_nonull[demo_nonull['DMDMARTZ'] != 77.0]
demo_nonull = demo_nonull[demo_nonull['DMDMARTZ'] != 99.0]
demo_nonull = demo_nonull[demo_nonull['DMDMARTZ'] != '.']
demo_nonull = demo_nonull[demo_nonull['DMDEDUC2'] != '.']
demo_nonull = demo_nonull[demo_nonull['DMDEDUC2'] != 7.0]
demo_nonull = demo_nonull[demo_nonull['DMDEDUC2'] != 9.0]

demo_clean = demo_nonull [['DMDMARTZ', 'DMDEDUC2']]

demo_clean


,DMDMARTZ,DMDEDUC2
0,1.0,5.0
1,1.0,5.0
2,1.0,3.0
6,3.0,2.0
7,1.0,3.0
...,...,...
11927,2.0,1.0
11929,3.0,5.0
11930,1.0,4.0
11931,2.0,4.0


In [115]:
# Simplify marriage results
dm_mar = demo_clean['DMDMARTZ'].astype(str)



dm_mar = dm_mar.replace('1.0', 'Married')
dm_mar = dm_mar.replace('2.0', 'Not Married')
dm_mar = dm_mar.replace('3.0', 'Not Married')
dm_mar.sample(10)
demo_clean
dm_mar
chisquare_marriage = dm_mar

,DMDMARTZ
0,Married
1,Married
2,Married
6,Not Married
7,Married
...,...
11927,Not Married
11929,Not Married
11930,Married
11931,Not Married


In [118]:
## Simplify Education Results
dm_edu = demo_clean['DMDEDUC2'].astype(str)


dm_edu = dm_edu.replace('1.0', 'No College')
dm_edu = dm_edu.replace('2.0', 'No College')
dm_edu = dm_edu.replace('3.0', 'No College')
dm_edu = dm_edu.replace('4.0', 'College')
dm_edu = dm_edu.replace('5.0', 'College')
dm_edu.sample(10)

dm_edu

chisquare_education = dm_edu

,DMDEDUC2
0,College
1,College
2,No College
6,No College
7,No College
...,...
11927,No College
11929,College
11930,College
11931,College


In [119]:
# Create Contingency Table
contingency_table = pd.crosstab(chisquare_marriage, chisquare_education)
print("contingency_table:")
print(contingency_table)

contingency_table:
DMDEDUC2     College  No College
DMDMARTZ                        
Married         2782        1352
Not Married     2207        1431


In [120]:
from scipy.stats import chi2_contingency

In [121]:
## First Simplified Version marriage vs education

stat, p_value, dof, expected = chi2_contingency(contingency_table)
print(f"\nChi-Square Test Statistic: {stat}, p-value: {p_value}")
print("Expected Frequencies:")
print(expected)

# Interpretation
if p_value > 0.05:
    print("\nNo significant association between Marriage and Education.")
else:
    print("\nSignificant association between Marriage and Education.")


Chi-Square Test Statistic: 36.72203920845173, p-value: 1.3623057407211713e-09
Expected Frequencies:
[[2653.69608852 1480.30391148]
 [2335.30391148 1302.69608852]]

Significant association between Marriage and Education.
